In [1]:
import pandas as pd
import pysam
import numpy as np
import akita_utils
import h5py

In [2]:
from akita_utils.dna_utils import scan_motif, dna_1hot

2023-05-31 16:52:17.028946: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/home1/smaruj/software/GSL/lib:/home1/smaruj/software/HTSLIB/lib
2023-05-31 16:52:17.029001: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [3]:
genome_file = "/project/fudenber_735/genomes/mm10/mm10.fa"

# reading tsv with all 1000 motifs

In [4]:
tsv_filepath = "/home1/smaruj/akita_utils/bin/insert_virtual_dots_vs_boundaries/boundaries_10k.tsv"
df = pd.read_table(tsv_filepath, sep="\t")

In [5]:
# adding seq_id

seq_id = []

nr_targets = 6
nr_sites = 1000
nr_backgrounds = 10
# since there are 1000 motifs and 10 backgrounds

for background_index in range(nr_backgrounds):
    for seq_index in range(nr_sites):
        seq_id.append(seq_index)

In [6]:
df["seq_id"] = seq_id

In [8]:
df_bg0 = df[df["background_index"] == 0]

In [10]:
# since the set of motifs is the same for all backgrounds, I'd focus on a subset of the table corresponding to the 0th background
df_bg0

,chrom,start,end,strand,genomic_SCD,orientation,background_index,flank_bp,spacer_bp,seq_id
0,chr12,35192361,35192380,+,53.780,<>,0,20,99980,0
1,chr2,48886252,48886271,-,53.700,<>,0,20,99980,1
2,chr12,35192359,35192378,-,53.660,<>,0,20,99980,2
3,chr8,102781112,102781131,-,53.500,<>,0,20,99980,3
4,chr5,49961991,49962010,-,53.300,<>,0,20,99980,4
...,...,...,...,...,...,...,...,...,...,...
995,chr2,118246115,118246134,-,4.390,<>,0,20,99980,995
996,chr5,117502801,117502820,+,4.367,<>,0,20,99980,996
997,chr15,76198798,76198817,+,4.363,<>,0,20,99980,997
998,chr5,143315378,143315397,-,4.360,<>,0,20,99980,998


In [11]:
# test
chrom = "chr12"
start = 35192361
end = 35192380
strand = "+"

In [20]:
with open("MA0139.1.jaspar") as f:
    for line in f.readlines():
        if line[0] in ["A", "C", "G", "T"]:
            print(line.split(sep="\t"))

['A  [    87    167    281     56      8    744     40    107    851      5    333     54     12     56    104    372     82    117    402 ]\n']
['C  [   291    145     49    800    903     13    528    433     11      0      3     12      0      8    733     13    482    322    181 ]\n']
['G  [    76    414    449     21      0     65    334     48     32    903    566    504    890    775      5    507    307     73    266 ]\n']
['T  [   459    187    134     36      2     91     11    324     18      3      9    341      8     71     67     17     37    396     59 ]\n']


In [ ]:
def scan_motif(seq_1hot, motif, strand=None):
    if motif.shape[-1] != 4:
        raise ValueError(
            "motif should be n_postions x 4 bases, A=0, C=1, G=2, T=3"
        )
    if seq_1hot.shape[-1] != 4:
        raise ValueError(
            "seq_1hot should be n_postions x 4 bases, A=0, C=1, G=2, T=3"
        )
    scan_forward = tf.nn.conv1d(
        np.expand_dims(seq_1hot, 0).astype(float),
        np.expand_dims(motif, -1).astype(float),
        stride=1,
        padding="SAME",
    ).numpy()[0]
    if strand == "forward":
        return scan_forward
    scan_reverse = tf.nn.conv1d(
        np.expand_dims(seq_1hot, 0).astype(float),
        np.expand_dims(hot1_rc(motif), -1).astype(float),
        stride=1,
        padding="SAME",
    ).numpy()[0]
    if strand == "reverse":
        return scan_reverse
    return np.maximum(scan_forward, scan_reverse).flatten()